# 役割
本番運用時の予測を行うnotebook。モデルの学習（`train.ipynb`）が完了している必要がある。

# インポート

In [1]:
import preprocessing
from feature_producing_prediction import PredictionFeatureCreator
import predictioning
%load_ext autoreload

In [86]:
%autoreload

# 事前準備
**当日出走馬が確定した時点**で実行できる

In [2]:
# 予測時の学習母集団はcommon/data/prediction_population/に配置している
POPULATION_DIR = preprocessing.COMMON_DATA_DIR / "prediction_population"

In [202]:
# 当日出走馬の過去成績テーブルの前処理
horse_results_preprocessed = preprocessing.process_horse_results(
    population_dir=POPULATION_DIR,
    populaton_filename="population.csv",
    input_filename="horse_results_prediction.csv",
    output_filename="horse_results_prediction.csv"
)

In [203]:
# データの重複チェック
horse_results_preprocessed.duplicated(subset=[ "horse_id", "date"]).sum()


np.int64(0)

In [204]:
# 当日出走馬の血統テーブルの前処理
peds_preprocessed = preprocessing.process_peds(
    population_dir=POPULATION_DIR,
    populaton_filename= "population.csv",
    input_filename="peds_prediction.csv",
    output_filename="peds_prediction.csv"
)

In [80]:
pfc = PredictionFeatureCreator()
# 過去成績集計は事前に行うことができる
pfc.create_baselog()
pfc.agg_horse_n_races()
pfc.agg_horse_n_races_relative()
pfc.agg_interval()

agg_horse_n_races_relative:   0%|          | 0/5 [00:00<?, ?it/s]

running agg_interval()...


In [206]:
pfc.baselog

,date,race_id,horse_id,date_horse,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,n_horses,time,win,rentai,show,place
2842,2025-03-02,202510011201,2022103128,2025-02-23,2.0,220.0,0.5,3.0,0,1700,0,1.0,12,106.1,0,1,1,10.0
2966,2025-03-02,202510011203,2022102890,2025-02-23,3.0,112.0,1.3,0.0,0,1000,0,1.0,14,60.4,0,0,1,10.0
273,2025-03-02,202506020207,2022101206,2025-02-23,10.0,0.0,1.5,0.0,1,1600,0,2.0,13,96.5,0,0,0,5.0
3411,2025-03-02,202510011206,2020100621,2025-02-23,12.0,0.0,1.0,1.0,1,1800,0,5.0,14,109.0,0,0,0,10.0
3437,2025-03-02,202510011206,2021106904,2025-02-23,5.0,80.0,0.8,1.0,1,2000,0,2.0,12,120.9,0,0,0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,2025-03-02,202506020211,2016106606,2018-11-24,11.0,0.0,1.3,0.0,1,1800,0,1.0,18,110.9,0,0,0,5.0
1107,2025-03-02,202506020211,2016106606,2018-11-11,10.0,0.0,1.0,0.0,1,1800,0,1.0,15,110.8,0,0,0,5.0
1108,2025-03-02,202506020211,2016106606,2018-11-04,8.0,0.0,2.6,1.0,0,1600,0,1.0,9,101.5,0,0,0,5.0
1109,2025-03-02,202506020211,2016106606,2018-09-22,13.0,0.0,5.2,1.0,0,1800,2,1.0,15,118.5,0,0,0,6.0


# 当日の予測処理
レース直前出走直前に実行する

In [82]:
pfc.race_info

,race_id,race_type,around,course_len,weather,ground_state,race_class,place
0,202509010210,0,0,1800,3,0,4,9


In [87]:
# 特徴量の更新
features = pfc.create_features(
    race_id="202509010210",  # 予測するレースidを指定
    skip_agg_horse=True  # 事前に集計した場合はスキップできる
)

fetching shubuta page html...


c:\Users\Onoe Daichi\Downloads\競馬AI\keiba_ai\v3_exp_4\src\feature_producing_prediction.py:184: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0]


agg_horse_per_course_len:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_ground_state_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_class:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

running agg_jockey()...
running agg_trainer()...
running agg_sire()...
running cross_feature()...
merging all features...


# race_classマッピングの修正

In [13]:
import json
from pathlib import Path
from bs4 import BeautifulSoup
# import numpy as np
import pandas as pd
import re
COMMON_DATA_DIR = Path("..", "..", "common", "data")
POPULATION_DIR = COMMON_DATA_DIR / "prediction_population"
MAPPING_DIR = COMMON_DATA_DIR / "mapping"
with open(MAPPING_DIR / "race_class.json", "r") as f:
    race_class_mapping = json.load(f)
# pfc.htmls["202509010210"]
# soup = BeautifulSoup(pfc.htmls["202509010210"], "lxml").find("div", class_="RaceList_Item02")
# soup

In [14]:
regex_race_class = "|".join(race_class_mapping)
regex_race_class

'新馬|未勝利|1勝クラス|１勝クラス|2勝クラス|２勝クラス|3勝クラス|３勝クラス|オープン|L|GIII|GII|GI|特別|500万下|1000万下|1600万下|OP'

In [7]:
df = pd.read_csv(COMMON_DATA_DIR/"rawdf"/"race_info_18-24.csv",sep="\t")[3630:3640]
df

,race_id,title,info_1,info_2
3630,201902010309,3歳以上1勝クラス,"['芝右1200m', '天候:雨', '芝:稍重', '発走:14:15']","['2019年6月22日', '1回函館3日目', '3歳以上1勝クラス', '牝', '指..."
3631,201902010310,湯川特別(1勝),"['芝右2000m', '天候:雨', '芝:稍重', '発走:14:50', '過去の湯川...","['2019年6月22日', '1回函館3日目', '3歳以上1勝クラス', '混', '特..."
3632,201902010311,大沼ステークス(L),"['ダ右1700m', '天候:雨', 'ダート:稍重', '発走:15:25', '過去の...","['2019年6月22日', '1回函館3日目', '3歳以上オープン', '国際', '特..."
3633,201902010312,長万部特別(1勝),"['芝右1200m', '天候:曇', '芝:稍重', '発走:16:05', '過去の長万...","['2019年6月22日', '1回函館3日目', '3歳以上1勝クラス', '特指', '..."
3634,201902010401,3歳未勝利,"['芝右1200m', '天候:曇', '芝:重', '発走:09:50']","['2019年6月23日', '1回函館4日目', '3歳未勝利', '混', '指', '..."
3635,201902010402,3歳未勝利,"['ダ右1700m', '天候:曇', 'ダート:重', '発走:10:15']","['2019年6月23日', '1回函館4日目', '3歳未勝利', '牝', '指', '..."
3636,201902010403,3歳未勝利,"['芝右1800m', '天候:曇', '芝:稍重', '発走:10:45']","['2019年6月23日', '1回函館4日目', '3歳未勝利', '指', '馬齢']"
3637,201902010404,3歳未勝利,"['ダ右1700m', '天候:曇', 'ダート:重', '発走:11:15']","['2019年6月23日', '1回函館4日目', '3歳未勝利', '指', '馬齢']"
3638,201902010405,2歳新馬,"['芝右1200m', '天候:曇', '芝:稍重', '発走:12:05']","['2019年6月23日', '1回函館4日目', '2歳新馬', '混', '指', '馬齢']"
3639,201902010406,3歳未勝利,"['芝右2000m', '天候:曇', '芝:稍重', '発走:12:35']","['2019年6月23日', '1回函館4日目', '3歳未勝利', '指', '馬齢']"


In [12]:
df["title"].str.extract(rf"({regex_race_class})").fillna(df["info_2"].str.extract(rf"({regex_race_class})"))[0]

3630    1勝クラス
3631       特別
3632        L
3633       特別
3634      未勝利
3635      未勝利
3636      未勝利
3637      未勝利
3638       新馬
3639      未勝利
Name: 0, dtype: object

In [15]:
df["race_class"] = (
        df["title"]
        .str.extract(rf"({regex_race_class})")
        # タイトルからレース階級情報が取れない場合はinfo2から取得
        .fillna(df["info_2"].str.extract(rf"({regex_race_class})"))[0]
        .map(race_class_mapping)
    )
df["race_class"]

3630    2
3631    5
3632    5
3633    5
3634    1
3635    1
3636    1
3637    1
3638    0
3639    1
Name: race_class, dtype: int64

In [66]:
divs = soup.find_all("div")
titile = soup.find("h1").text.strip()
regex_race_class = "|".join(race_class_mapping)
race_class_title = re.findall(regex_race_class, titile)
race_class_title

[]

In [68]:
# regex_race_class = "|".join(race_class_mapping)
race_class = re.findall(regex_race_class, divs[1].text)
race_class

['３勝クラス']

In [ ]:
info_dict = {}
if len(race_class_title) != 0:
            info_dict = race_class_mapping[race_class_title[0]]
elif len(race_class) != 0:
            info_dict = race_class_mapping[race_class[0]]
else:
        info_dict = None

In [85]:
info_dict

4

In [ ]:
clean_text = re.sub(r"\s+", "", divs[1].text) 
clean_text

'1回阪神2日目サラ系４歳以上３勝クラス(混)(特指)定量10頭本賞金:1840,740,460,280,184万円'

In [83]:
# 予測
prediction = predictioning.predict(
    features,
    model_filename="model_latest.pkl",
    calibration_model_filename="calibration_model_latest.pkl",
    # sort_col="expect_return",
)

In [84]:
# 2024年までを学習させたバージョン
sorted_features = prediction.sort_values(by='expect_return', ascending=False)
sorted_features

,race_id,umaban,tansyo_odds,popularity,pred,pred_calibrated,expect_return,expect_return_calibrated
0,202509010210,1,54.7,8,0.018806,0.020367,1.028703,1.114053
6,202509010210,7,35.2,6,0.025132,0.025510,0.884644,0.897959
2,202509010210,3,1.4,1,0.627014,0.903430,0.877819,1.264802
8,202509010210,9,12.9,3,0.066229,0.043140,0.854360,0.556506
5,202509010210,6,13.7,4,0.059661,0.043140,0.817353,0.591018
1,202509010210,2,33.6,5,0.023875,0.025510,0.802208,0.857143
9,202509010210,10,80.9,9,0.009461,0.005450,0.765419,0.440872
4,202509010210,5,36.9,7,0.020019,0.020367,0.738691,0.751527
7,202509010210,8,3.5,2,0.209192,0.241379,0.732173,0.844828
3,202509010210,4,156.9,10,0.004560,0.004231,0.715542,0.663893
